# Scraping Data Hotel from Pegi-pegi

link for [Pegi-pegi](https://www.pegipegi.com/)

In [42]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [150]:
# get url for each page
def list_url(city):
    url_list = [f'https://www.pegipegi.com/hotel/{city}/']
    for i in range(2,75):
        url = f'https://www.pegipegi.com/hotel/{city}/'  + str(i)+'.html/'
        url_list.append(url)
    return url_list

In [157]:
# Search hotel
def cari_hotel(list_url):    
    name_all = []
    image_all=[]
    city_all = []
    prov_all = []
    price_all = []
    star_all = []
    rating_all = []
    review_all = []
    desc_all = []
    for i in list_url:
        html = requests.get(i)
        soup = BeautifulSoup(html.content, 'html.parser')
        content = soup.find_all('div', {'class':"listContent" ,'itemprop':"itemListElement"})
        
        # find name
        title = soup.find_all('div',attrs={'class':'title'})
        name=[]
        for j in title[1:-8]:
            name.append(j.text.replace('\n','')[1:-1])
        name_all += name

        #find image
        content_img = soup.find_all('div', {'class':"contentImage"})
        image=[]
        for i in content_img[:3]:
            image.append(i.find('img')['src'])
        for i in content_img[3:]:
            image.append(i.find('img')['data-original'])
        image_all += image

        # find city and province
        city=[]
        prov=[]
        for i in content:
            address = i.find('div',attrs={'class':'address'}).text.replace('\n','').replace('    Tampilkan Peta','')
            city.append(address.split(',')[0][2:])
            prov.append(address.split(',')[1][4:])
        city_all += city
        prov_all += prov

        # find price
        price =[]
        for i in content:
            price.append(i.find('div', {'class':"diskonPrice"}).text.replace('\n','')[50:-44])
        price_all += price

        # find star
        star = []
        for i in content:
            try :
                star_ = len(i.find_all('i', {'class':"fa fa-star"}))
            except :
                star_ = np.NaN
            star.append(star_)
        star_all += star
        
        # find rating
        rating=[]
        for i in content:
            try :
                rat = i.find('div', {'class':"ratingRight"}).find('span').text
            except :
                rat = np.NaN
            rating.append(rat)
        rating_all += rating
        
        #find review
        review=[]
        for i in content:
            try :
                rev = i.find('div', {'class':"ratingRight"}).find_all('span')[2].text[1:-1]
            except :
                rev = np.NaN
            review.append(rev)
        review_all += review
        
        #description
        desc=[]
        for i in content:
            try :
                descrip = i.find('div', {'class':"contentDescriptionHotel"}).find('p').text
            except :
                descrip = np.NaN
            desc.append(descrip)
        desc_all += desc
    
    df = pd.DataFrame({'Name': name_all,
                  'Image': image_all,
                  'City': city_all,
                  'Province': prov_all,
                  'Price': price_all,
                  'Star' : star_all,     
                  'Rating' : rating_all,
                  'Review' : review_all,
                  'Description' : desc_all})
    return df

In [158]:
kota = ['jakarta','bali','bandung','bogor','surabaya','malang','semarang','medan','jogja','solo']
daftar_url = []
for i in kota:
    x = list_url(i)
    daftar_url += x
len(daftar_url)

740

In [159]:
all_hotel = cari_hotel(daftar_url)
all_hotel.head()

,Name,Image,City,Province,Price,Star,Rating,Review,Description
0,Hotel Horison Ciledug Jakarta,https://origin.pegipegi.com/jalan/images/pictS...,Jakarta Selatan,Jakarta,467.682,3,9.2,115,"Sebuah hunian yang nyaman, bersih, dan desain ..."
1,Maple Hotel Grogol,https://origin.pegipegi.com/jalan/images/pictS...,Jakarta Barat,Jakarta,343.500,3,8.6,105,"Sebuah hunian yang nyaman, bersih, dan desain ..."
2,Hotel Menara Peninsula,https://origin.pegipegi.com/jalan/images/pictS...,Jakarta Barat,Jakarta,450.000,4,8.7,57,"This cosmopolitan hotel, situated in the centr..."
3,Kyriad Hotel Metro Kebayoran,https://origin.pegipegi.com/jalan/images/pictS...,Jakarta Selatan,Jakarta,366.676,3,8.8,139,Sebuah penginapan berbentuk hotel yang nyaman ...
4,Rasuna Icon Jakarta,https://origin.pegipegi.com/jalan/images/pictS...,Jakarta Selatan,Jakarta,344.750,3,8.5,28,"Dirancang untuk bisnis maupun liburan, Rasuna ..."


In [160]:
all_hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5658 entries, 0 to 5657
Data columns (total 9 columns):
Name           5658 non-null object
Image          5658 non-null object
City           5658 non-null object
Province       5658 non-null object
Price          5658 non-null object
Star           5658 non-null int64
Rating         2483 non-null object
Review         2327 non-null object
Description    5658 non-null object
dtypes: int64(1), object(8)
memory usage: 398.0+ KB


In [161]:
all_hotel['Star'].replace(0,np.NaN, inplace=True)
all_hotel.sort_values(by=['Province','City','Name'], inplace=True)
all_hotel.reset_index(inplace=True,drop=True)
all_hotel

,Name,Image,City,Province,Price,Star,Rating,Review,Description
0,100 Sunset Boutique,https://origin.pegipegi.com/jalan/images/pictS...,Badung,Bali,1.400.000,3.0,7.0,5,"100 Sunset Boutique, terletak di Jalan Sunset ..."
1,100 Sunset Hotel,https://origin.pegipegi.com/jalan/images/pictS...,Badung,Bali,900.000,3.0,NaN,NaN,Akomodasi yang tak kalah nyaman. 100 Sunset Ho...
2,18 Suite Villa Loft Kuta by AMITHYA,https://origin.pegipegi.com/jalan/images/pictS...,Badung,Bali,2.800.000,4.0,8.7,6,Terletak di daerah terpencil namun masih dalam...
3,7 Bidadari Seminyak,https://origin.pegipegi.com/jalan/images/pictS...,Badung,Bali,270.000,3.0,8.2,4,7 Bidadari Seminyak adalah sebuah hotel butik ...
4,7 Chakras Resort,https://origin.pegipegi.com/jalan/images/pictS...,Badung,Bali,2.800.000,5.0,NaN,NaN,7 Chakras Resort terletak di tepi Seminyak dan...
...,...,...,...,...,...,...,...,...,...
5653,STYLE 50 Homestay,https://origin.pegipegi.com/jalan/images/pictS...,Wonokromo,Surabaya,225.000,NaN,9.0,8,Akomodasi yang tak kalah nyaman dan terjangkau...
5654,Studio Room at Tamansari Papilio Apartment Sur...,https://d1nabgopwop1kh.cloudfront.net/v2/hotel...,Wonokromo,Surabaya,355.000,NaN,NaN,NaN,
5655,Studio Room at Tamansari Papilio Surabaya (REN),https://d1nabgopwop1kh.cloudfront.net/v2/hotel...,Wonokromo,Surabaya,312.500,NaN,NaN,NaN,
5656,Studio Room for 3 at Papilio Apartment by HUM'Z,https://d1nabgopwop1kh.cloudfront.net/v2/hotel...,Wonokromo,Surabaya,299.000,NaN,NaN,NaN,


In [162]:
all_hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5658 entries, 0 to 5657
Data columns (total 9 columns):
Name           5658 non-null object
Image          5658 non-null object
City           5658 non-null object
Province       5658 non-null object
Price          5658 non-null object
Star           3881 non-null float64
Rating         2483 non-null object
Review         2327 non-null object
Description    5658 non-null object
dtypes: float64(1), object(8)
memory usage: 398.0+ KB


In [166]:
#save as csv
all_hotel.to_csv('hotel_pegi.csv', index=False)